In [1]:
import warnings
from pathlib import Path

import cartopy.crs as ccrs
import iris
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
from tqdm.notebook import tqdm

In [2]:
from aeolus.coord import add_cyclic_point_to_cube, replace_z_coord
from aeolus.plot import subplot_label_generator

In [3]:
from util_commons import EXPERIMENTS, GASES, LAYERS, SEASONS
from util_mypaths import path_to_processed, path_to_results
from util_plot_func import m_to_km, use_paper_style
from util_proc_func import calc_mean_diff_pval, mmr_to_vmr_in_ppn, stipple_out

In [4]:
warnings.filterwarnings("ignore", module="iris")
use_paper_style()

In [5]:
# List of chemical species to consider
SPECIES = [
    #     "ch4",
    #     "c2h6",
    #     "c3h8",
    "no",
    "no2",
    "hono",
    "ho2no2",
    "hno3",
    "n2o5",
    "pan",
    "ppan",
    "meono2",
    "etono2",
    "nprono2",
    "iprono2",
    "o3",
]

In [6]:
# Switches
w_rono2 = True
xy = True
yz = False

In [7]:
# Read data
vrbls = {}
for exp in EXPERIMENTS.keys():
    vrbls[exp] = {}
    for sp in SPECIES:
        cube_in_mmr = iris.load_cube(
            str(path_to_processed / EXPERIMENTS[exp] / f"{EXPERIMENTS[exp]}_{sp}.nc"), sp
        )
        # Convert mass mixing ratio to volume mixing ratio
        cube_in_vmr = mmr_to_vmr_in_ppn(cube_in_mmr, GASES[sp]["molar_mass"], GASES[sp]["ppn"])
        # Replace model_level_number coordinate with level_height coordinate
        cube_on_hlevs = replace_z_coord(cube_in_vmr)
        cube_on_hlevs.rename(sp)
        cube_on_hlevs.units = GASES[sp]["ppn"]
        # Remove the first year as a spin up
        cube_on_hlevs = cube_on_hlevs[12::, ...]
        # Add season and year coordinates
        iris.coord_categorisation.add_season(cube_on_hlevs, "time", name="season")
        iris.coord_categorisation.add_season_year(cube_on_hlevs, "time", name="year")
        vrbls[exp][sp] = cube_on_hlevs

# Calculate NOx and NOy
for exp in EXPERIMENTS.keys():
    nox = vrbls[exp]["no"] + vrbls[exp]["no2"]
    noy = (
        nox
        + vrbls[exp]["hono"]
        + vrbls[exp]["ho2no2"]
        + vrbls[exp]["hno3"]
        + vrbls[exp]["n2o5"]
        + vrbls[exp]["pan"]
        + vrbls[exp]["ppan"]
    )
    # Include RONO2 into NOy?
    if w_rono2:
        rono2 = (
            vrbls[exp]["meono2"]
            + vrbls[exp]["etono2"]
            + vrbls[exp]["nprono2"]
            + vrbls[exp]["iprono2"]
        )
        rono2.convert_units("ppbv")
        noy = noy + rono2
    nox.rename("nox")
    noy.rename("noy")
    nox.units = "ppbv"
    noy.units = "ppbv"
    vrbls[exp]["nox"] = nox
    vrbls[exp]["noy"] = noy

### Layer seasonal mean maps

In [8]:
# Plotting parameters for each variable
var_kws = {
    "o3": {
        "ssn_mean_diff_abs_levs": np.arange(-15, 16, 1),  # bl np.arange(-10, 12, 2),
        "ssn_mean_diff_pct_levs": np.arange(-100, 110, 10),  # bl np.arange(-40, 50, 10),
        "to_pptv": False,
        "ssn_mean_levs": np.arange(0, 130, 10),  # bl np.arange(0, 110, 10)
        "ssn_mean_levs_scale": "linear",
    },
    "nox": {
        "ssn_mean_diff_abs_levs": np.arange(-1.25, 1.50, 0.25),
        "ssn_mean_diff_pct_levs": np.arange(-200, 220, 20),
        "to_pptv": False,
        "ssn_mean_levs": np.arange(-4, 3, 1),
        "ssn_mean_levs_scale": "log",
        "cbar0_xticklabels": [
            "$10^{-4}$",
            "$10^{-3}$",
            "$10^{-2}$",
            "$10^{-1}$",
            "$10^0$",
            "$10^1$",
            "$10^2$",
        ],
    },
    "hono": {
        "ssn_mean_diff_abs_levs": np.arange(-2, 2.5, 0.5),
        "ssn_mean_diff_pct_levs": np.arange(-300, 350, 50),
        "to_pptv": True,
        "ssn_mean_levs": np.arange(0, 12, 1),
        "ssn_mean_levs_scale": "linear",
    },
    "ho2no2": {
        "ssn_mean_diff_abs_levs": np.arange(-5, 6, 1),
        "ssn_mean_diff_pct_levs": np.arange(-300, 350, 50),
        "to_pptv": True,
        "ssn_mean_levs": np.arange(0.001, 45, 5),  # if min=0, plots incorrectly
        "ssn_mean_levs_scale": "linear",
    },
    "hno3": {
        "ssn_mean_diff_abs_levs": np.arange(-1.5, 2, 0.5),
        "ssn_mean_diff_pct_levs": np.arange(-200, 250, 50),
        "to_pptv": False,
        "ssn_mean_levs": np.arange(-3, 2, 1),
        "ssn_mean_levs_scale": "log",
        "cbar0_xticklabels": ["$10^{-3}$", "$10^{-2}$", "$10^{-1}$", "$10^0$", "$10^1$"],
    },
    "n2o5": {
        "ssn_mean_diff_abs_levs": np.arange(-12, 14, 2),
        "ssn_mean_diff_pct_levs": np.arange(-1200, 1400, 200),
        "to_pptv": True,
        "ssn_mean_levs": np.arange(0, 55, 5),
        "ssn_mean_levs_scale": "linear",
    },
    "pan": {
        "ssn_mean_diff_abs_levs": np.arange(-0.3, 0.4, 0.1),
        "ssn_mean_diff_pct_levs": np.arange(-100, 110, 10),
        "to_pptv": False,
        "ssn_mean_levs": np.arange(-4, 2, 1),
        "ssn_mean_levs_scale": "log",
        "cbar0_xticklabels": [
            "$10^{-4}$",
            "$10^{-3}$",
            "$10^{-2}$",
            "$10^{-1}$",
            "$10^0$",
            "$10^1$",
        ],
    },
    "ppan": {
        "ssn_mean_diff_abs_levs": np.arange(-3, 4, 1),
        "ssn_mean_diff_pct_levs": np.arange(-200, 250, 50),
        "to_pptv": True,
        "ssn_mean_levs": np.arange(0, 35, 5),
        "ssn_mean_levs_scale": "linear",
    },
}

In [9]:
def plt_ssn_maps(var, layer, ctrl_dict, sens_dict):
    """Plot seasonal mean distribution of a chosen variable in a chosen layer.

    (Left column) shows the distribution in the control experiment,
    (middle column) absolute difference between the sensitivity and control experiment,
    (right column) relative difference between the sensitivity and control experiment.
    Stippling shows the areas where the difference is not statistically significant."""
    iletters = subplot_label_generator()

    lons = ctrl_dict[var]["djf"]["ssn_mean"].coord("longitude").points
    lats = ctrl_dict[var]["djf"]["ssn_mean"].coord("latitude").points

    map_kws = dict(transform=ccrs.PlateCarree())
    cax_kw = dict(orientation="horizontal")
    ssn_mean_kws = dict(cmap="viridis", levels=var_kws[var]["ssn_mean_levs"], **map_kws)
    ssn_mean_diff_abs_kws = dict(
        cmap="RdBu_r", levels=var_kws[var]["ssn_mean_diff_abs_levs"], **map_kws
    )
    ssn_mean_diff_pct_kws = dict(
        cmap="RdBu_r", levels=var_kws[var]["ssn_mean_diff_pct_levs"], **map_kws
    )
    pval_kws = dict(s=1, c="grey", marker=".", alpha=1, **map_kws)

    fig, ax = plt.subplots(
        nrows=4,
        ncols=3,
        figsize=(20, 20),
        subplot_kw=dict(projection=ccrs.Robinson(central_longitude=180)),
        facecolor="w",
    )
    for ssn, iax in zip(SEASONS.keys(), ax[:, 0].flatten()):
        if var_kws[var]["ssn_mean_levs_scale"] == "linear":
            ctrl_ssn_mean = ctrl_dict[var][ssn]["ssn_mean"]
            ppn = GASES[var]["ppn"]
            if var_kws[var]["to_pptv"]:
                ctrl_ssn_mean = ctrl_ssn_mean * 1e3
                ppn = "pptv"
            p0 = iax.contourf(lons, lats, ctrl_ssn_mean.data, **ssn_mean_kws)
        if var_kws[var]["ssn_mean_levs_scale"] == "log":
            ctrl_ssn_mean = iris.analysis.maths.log10(
                ctrl_dict[var][ssn]["ssn_mean"], in_place=False
            )
            ppn = GASES[var]["ppn"]
            if var_kws[var]["to_pptv"]:
                ctrl_ssn_mean = ctrl_ssn_mean * 1e3
                ppn = "pptv"
            p0 = iax.contourf(
                lons,
                lats,
                ctrl_ssn_mean.data,
                **ssn_mean_kws,
            )
        if iax.is_first_row():
            iax.set_title("CTRL", loc="center", weight="bold")
        if iax.is_first_col():
            iax.annotate(
                f"         {ssn.upper()}", xy=(0, 1), xycoords="axes fraction", weight="bold"
            )
    for ssn, iax in zip(SEASONS.keys(), ax[:, 1].flatten()):
        sens_ssn_mean = sens_dict[var][ssn]["ssn_mean_diff_abs"]
        if var_kws[var]["to_pptv"]:
            sens_ssn_mean = sens_ssn_mean * 1e3
        p1 = iax.contourf(lons, lats, sens_ssn_mean.data, **ssn_mean_diff_abs_kws)
        iax.scatter(*stipple_out(sens_dict[var][ssn]["pval"]), **pval_kws)

    for ssn, iax in zip(SEASONS.keys(), ax[:, 2].flatten()):
        p2 = iax.contourf(
            lons, lats, sens_dict[var][ssn]["ssn_mean_diff_pct"].data, **ssn_mean_diff_pct_kws
        )
        iax.scatter(*stipple_out(sens_dict[var][ssn]["pval"]), **pval_kws)
    for iax in ax[0, 1:3].flatten():
        iax.set_title(f"{sens} - CTRL", loc="center", weight="bold")
    for iax in ax.flatten():
        iax.set_title(f"({next(iletters)})", loc="left")
        iax.set_global()
        iax.coastlines()
    fig.subplots_adjust(hspace=-0.6, wspace=0.02)
    cax0 = fig.add_axes([0.185, 0.22, 0.135, 0.005])  # [left, bottom, width, height]
    cax1 = fig.add_axes([0.446, 0.22, 0.135, 0.005])
    cax2 = fig.add_axes([0.706, 0.22, 0.135, 0.005])
    cbar0 = fig.colorbar(p0, cax=cax0, label=f"{GASES[var]['tex']} [{ppn}]", **cax_kw)
    if var_kws[var]["ssn_mean_levs_scale"] == "log":
        cbar0.ax.set_xticklabels(var_kws[var]["cbar0_xticklabels"])
    fig.colorbar(p1, cax=cax1, label=f"$\Delta${GASES[var]['tex']} [{ppn}]", **cax_kw)
    fig.colorbar(p2, cax=cax2, label=f"$\Delta${GASES[var]['tex']} [%]", **cax_kw)
    fig.suptitle(LAYERS[layer]["tex"], x=0.5125, y=0.8, weight="bold")
    fig.savefig(
        path_to_results
        / "xojn"
        / "ssn_mean_xy"
        / var
        / f"{sens.lower()}_{'CTRL'.lower()}_{var}_ssn_mean_xy_{layer}.png"
    )

In [10]:
if xy:
    # Choose atmospheric layer
    layer = "0-2km"  # 0-2km 5-10km 0-17km 0-85km

    # Calculate 9-year-average seasonal mean in a chosen atmospheric layer
    VRBLS = {}
    for exp in EXPERIMENTS.keys():
        VRBLS[exp] = {}
        for var in vrbls[exp].keys():
            VRBLS[exp][var] = {}
            for ssn in SEASONS.keys():
                # Extract a time series for each latitude-longitude within a layer by season
                # and average each time slice in height
                ssn_ts = (
                    vrbls[exp][var]
                    .extract(LAYERS[layer]["iris_constr"] & SEASONS[ssn]["iris_constr"])
                    .collapsed(["level_height"], iris.analysis.MEAN)
                )
                # Calculate 9-year-average seasonal mean
                _ssn_mean = ssn_ts.collapsed("year", iris.analysis.MEAN)
                # Add a cyclic point for plotting
                ssn_mean = add_cyclic_point_to_cube(_ssn_mean)
                # Find minimum and maximum
                ssn_mean_min, ssn_mean_max = ssn_mean.data.min(), ssn_mean.data.max()
                # Assemble data
                VRBLS[exp][var][ssn] = {
                    "ssn_ts": ssn_ts,
                    "ssn_mean": ssn_mean,
                    "ssn_mean_min": ssn_mean_min,
                    "ssn_mean_max": ssn_mean_max,
                }

    # Calculate seasonal mean differences and their statistical significance, and find extrema
    for sens in [k for k in EXPERIMENTS.keys() if k != "CTRL"]:
        for var in tqdm(VRBLS["CTRL"].keys()):
            for ssn in SEASONS.keys():
                ssn_mean_diff_abs = (
                    VRBLS[sens][var][ssn]["ssn_mean"] - VRBLS["CTRL"][var][ssn]["ssn_mean"]
                )
                ssn_mean_diff_pct = ssn_mean_diff_abs / VRBLS["CTRL"][var][ssn]["ssn_mean"] * 100
                ssn_mean_diff_abs_min, ssn_mean_diff_abs_max = (
                    ssn_mean_diff_abs.data.min(),
                    ssn_mean_diff_abs.data.max(),
                )
                ssn_mean_diff_pct_min, ssn_mean_diff_pct_max = (
                    ssn_mean_diff_pct.data.min(),
                    ssn_mean_diff_pct.data.max(),
                )
                pval = calc_mean_diff_pval(
                    VRBLS["CTRL"][var][ssn]["ssn_ts"], VRBLS[sens][var][ssn]["ssn_ts"]
                )
                # Assemble data
                VRBLS[sens][var][ssn] = {
                    "ssn_mean_diff_abs": ssn_mean_diff_abs,
                    "ssn_mean_diff_pct": ssn_mean_diff_pct,
                    "pval": pval,
                    "ssn_mean_diff_abs_min": ssn_mean_diff_abs_min,
                    "ssn_mean_diff_abs_max": ssn_mean_diff_abs_max,
                    "ssn_mean_diff_pct_min": ssn_mean_diff_pct_min,
                    "ssn_mean_diff_pct_max": ssn_mean_diff_pct_max,
                }

    # Find common extrema in the original units (see GASES[sp]["ppn"]) for plotting
    var_lims = {}
    for var in vrbls[exp].keys():
        var_lims[var] = {}
        ssn_mean_min_com = []
        ssn_mean_max_com = []
        ssn_mean_diff_abs_min_com = []
        ssn_mean_diff_abs_max_com = []
        ssn_mean_diff_pct_min_com = []
        ssn_mean_diff_pct_max_com = []
        for ssn in SEASONS.keys():
            ssn_mean_min_com.append(VRBLS["CTRL"][var][ssn]["ssn_mean_min"])
            ssn_mean_max_com.append(VRBLS["CTRL"][var][ssn]["ssn_mean_max"])
            for sens in [k for k in EXPERIMENTS.keys() if k != "CTRL"]:
                ssn_mean_diff_abs_min_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_abs_min"])
                ssn_mean_diff_abs_max_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_abs_max"])
                ssn_mean_diff_pct_min_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_pct_min"])
                ssn_mean_diff_pct_max_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_pct_max"])
        var_lims[var] = {
            "ssn_mean_min_com": np.min(ssn_mean_min_com),
            "ssn_mean_max_com": np.max(ssn_mean_max_com),
            "ssn_mean_diff_abs_min_com": np.min(ssn_mean_diff_abs_min_com),
            "ssn_mean_diff_abs_max_com": np.max(ssn_mean_diff_abs_max_com),
            "ssn_mean_diff_pct_min_com": np.min(ssn_mean_diff_pct_min_com),
            "ssn_mean_diff_pct_max_com": np.max(ssn_mean_diff_pct_max_com),
        }

#     # Plot
#     ctrl_dict = VRBLS["CTRL"]
#     for sens in [k for k in EXPERIMENTS.keys() if k != "CTRL"]:
#         for var in var_kws.keys():
#             sens_dict = VRBLS[sens]
#             plt_ssn_maps(var, layer, ctrl_dict, sens_dict)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

/home/maria/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: UserWarning: Warning: converting a masked element to nan.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [11]:
S_of_60S = iris.Constraint(latitude=lambda cell: -90.0 <= cell <= -60.0)

In [12]:
for var in var_kws.keys():
    if var_kws[var]["to_pptv"] or var in["hno3", "pan"]:
        factor = 1e3
    else:
        factor = 1
    print(var)
    for ssn in SEASONS.keys():
        print(
            ssn,
            VRBLS["CTRL"][var][ssn]["ssn_mean"].extract(S_of_60S).data.mean()*factor,
            VRBLS[sens][var][ssn]["ssn_mean_diff_abs"].extract(S_of_60S).data.max()*factor,
            VRBLS[sens][var][ssn]["ssn_mean_diff_pct"].extract(S_of_60S).data.max(),
        )

o3
djf 9.298796653747559 2.1685028076171875 26.98376
mam 13.77204704284668 1.1365184783935547 7.452431
jja 19.081926345825195 0.7477588653564453 3.722326
son 15.86756420135498 1.3486137390136719 8.406532
nox
djf 0.0018675290048122406 0.0066558122634887695 188.5471
mam 0.0013496128376573324 0.0007413404528051615 63.940094
jja 0.0018423418514430523 0.0005016792565584183 11.206623
son 0.0018111116951331496 0.0015040308935567737 99.870255
hono
djf 0.0038764201235608198 0.03499537706375122 274.22382
mam 0.0006128916538727935 0.0008136653377732728 109.2427
jja 0.0010481420531505137 0.00108601034298772 21.758652
son 0.0012746496622639825 0.001952392722159857 170.13582
ho2no2
djf 0.27354463236406446 0.9126272052526474 257.18454
mam 0.12234177847858518 0.31362372101284564 145.19765
jja 0.10385004134150222 0.06290763849392533 28.097143
son 0.5841344245709479 1.158969011157751 124.108864
hno3
djf 7.46601726859808 12.342751026153564 156.14136
mam 3.4897816367447376 2.447707112878561 60.413315
jja 

In [13]:
# 5-10km
# nox
# djf 0.010512636043131351 0.002041342668235302 37.583477
# mam 0.002143938560038805 0.0006801469717174768 39.728264
# jja 0.0009242983651347458 0.0003490210510790348 52.74728
# son 0.004653060808777809 0.0017694910056889057 41.094456
# hono
# djf 0.013769402357866056 0.004244857336743735 49.454815
# mam 0.0020312390915933065 0.0007001171979936771 40.547897
# jja 0.0008490382583659084 0.0003410295903449878 33.515556
# son 0.0048496654017071705 0.0025215172172465827 53.918594
# ho2no2
# djf 2.598260762169957 0.6181842181831598 37.64899
# mam 1.4401567168533802 0.30205852817744017 27.95187
# jja 1.2308578006923199 0.2598509890958667 18.653244
# son 2.0651591476053 0.6907163187861443 36.94521
# hno3
# djf 56.728243827819824 4.999976605176926 12.909156
# mam 64.47827070951462 10.291196405887604 14.003921
# n2o5
# djf 0.039718011976219714 0.017759070033207536 37.022255
# mam 0.2371600567130372 0.1383290800731629 57.85004
# jja 0.23763799981679767 0.06235041655600071 25.391779
# son 0.03416374238440767 0.0191817780432757 67.20726

In [14]:
break

SyntaxError: 'break' outside loop (<ipython-input-14-6aaf1f276005>, line 4)

### Layer seasonal mean zonal cross sections

In [ ]:
def plt_ssn_cross_sections(var, layer, ctrl_dict, sens_dict):
    """Plot zonal seasonal mean distribution of a chosen variable in a chosen layer.

    (Left column) shows the distribution in the control experiment,
    (middle column) absolute difference between the sensitivity and control experiment,
    (right column) relative difference between the sensitivity and control experiment.
    Stippling shows the areas where the difference is not statistically significant."""
    iletters = subplot_label_generator()

    levs = ctrl_dict[var]["djf"]["ssn_mean"].coord("level_height").points
    lats = ctrl_dict[var]["djf"]["ssn_mean"].coord("latitude").points

    if var == "o3":
        if layer in ["0-85km"]:
            ssn_mean_levs = np.arange(0, 12000, 1000)
            ssn_mean_diff_abs_levs = np.arange(-1000, 1200, 200)
            ssn_mean_diff_pct_levs = np.arange(-50, 60, 10)
        elif layer == "0-17km":
            ssn_mean_levs = np.arange(0, 130, 10)
            ssn_mean_diff_abs_levs = np.arange(-5, 5.5, 0.5)
            ssn_mean_diff_pct_levs = np.arange(-20, 22, 2)
    elif var == "nox":
        if layer in ["0-85km"]:
            ssn_mean_levs = np.arange(-6, 3, 1)
            ssn_mean_diff_abs_levs = np.arange(-1.25, 1.50, 0.25)
            ssn_mean_diff_pct_levs = np.arange(-250, 300, 50)
            cbar_xticklbls = [
                "$10^{-6}$",
                "$10^{-5}$",
                "$10^{-4}$",
                "$10^{-3}$",
                "$10^{-2}$",
                "$10^{-1}$",
                "$10^0$",
                "$10^1$",
                "$10^2$",
            ]
        elif layer == "0-17km":
            ssn_mean_levs = np.arange(-6, 1.5, 0.5)
            ssn_mean_diff_abs_levs = np.arange(-0.05, 0.06, 0.01)
            ssn_mean_diff_pct_levs = np.arange(-200, 220, 20)
            cbar_xticklbls = [
                "$10^{-6}$",
                "$10^{-5}$",
                "$10^{-4}$",
                "$10^{-3}$",
                "$10^{-2}$",
                "$10^{-1}$",
                "$10^0$",
                "$10^1$",
            ]

    cax_kw = dict(orientation="horizontal")
    ssn_mean_kws = dict(cmap="viridis", levels=ssn_mean_levs, extend="max")
    ssn_mean_diff_abs_kws = dict(cmap="RdBu_r", levels=ssn_mean_diff_abs_levs, extend="both")
    ssn_mean_diff_pct_kws = dict(cmap="RdBu_r", levels=ssn_mean_diff_pct_levs, extend="both")
    pval_kws = dict(s=5, c="grey", marker=".", alpha=1)

    fig, ax = plt.subplots(
        nrows=4, ncols=3, figsize=(20, 20), facecolor="w", sharex=True, sharey=True
    )
    for ssn, iax in zip(SEASONS.keys(), ax[:, 0].flatten()):
        if var == "o3":
            p0 = iax.contourf(lats, levs, ctrl_dict[var][ssn]["ssn_mean"].data, **ssn_mean_kws)
        else:
            ctrl_ssn_mean = iris.analysis.maths.log10(
                ctrl_dict[var][ssn]["ssn_mean"], in_place=False
            )
            p0 = iax.contourf(
                lats,
                levs,
                ctrl_ssn_mean.data,
                **ssn_mean_kws,
            )
        if iax.is_first_row():
            iax.set_title("CTRL", loc="center", weight="bold")
        iax.annotate(f"      {ssn.upper()}", xy=(0, 1.025), xycoords="axes fraction", weight="bold")
        iax.set_ylabel("Height [km]")

    for ssn, iax in zip(SEASONS.keys(), ax[:, 1].flatten()):
        p1 = iax.contourf(
            lats,
            levs,
            np.ma.masked_where(
                ctrl_dict["o3"][ssn]["ssn_mean"].data >= 120,
                sens_dict[var][ssn]["ssn_mean_diff_abs"].data,
            ),
            **ssn_mean_diff_abs_kws,
        )
        iax.scatter(*stipple_out(sens_dict[var][ssn]["pval"]), **pval_kws)

    for ssn, iax in zip(SEASONS.keys(), ax[:, 2].flatten()):
        p2 = iax.contourf(
            lats,
            levs,
            np.ma.masked_where(
                ctrl_dict["o3"][ssn]["ssn_mean"].data >= 120,
                sens_dict[var][ssn]["ssn_mean_diff_pct"].data,
            ),
            **ssn_mean_diff_pct_kws,
        )
        iax.scatter(*stipple_out(sens_dict[var][ssn]["pval"]), **pval_kws)

    for iax in ax[0, 1:3].flatten():
        iax.set_title(f"{sens} - CTRL", loc="center", weight="bold")
    for iax in ax.flatten():
        iax.set_title(f"({next(iletters)})", loc="left")
        iax.yaxis.set_major_formatter(FuncFormatter(m_to_km))
    for iax in ax[3, :].flatten():
        iax.set_xlabel("Latitude")
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    cax0 = fig.add_axes([0.185, 0.09, 0.135, 0.005])  # [left, bottom, width, height]
    cax1 = fig.add_axes([0.446, 0.09, 0.135, 0.005])
    cax2 = fig.add_axes([0.706, 0.09, 0.135, 0.005])
    cbar0 = fig.colorbar(p0, cax=cax0, label=f"{GASES[var]['tex']} [{GASES[var]['ppn']}]", **cax_kw)
    if var == "nox":
        cbar0.ax.set_xticklabels(cbar_xticklbls)
    fig.colorbar(p1, cax=cax1, label=f"$\Delta${GASES[var]['tex']} [{GASES[var]['ppn']}]", **cax_kw)
    fig.colorbar(p2, cax=cax2, label=f"$\Delta${GASES[var]['tex']} [%]", **cax_kw)
    fig.suptitle(LAYERS[layer]["tex"], x=0.5125, y=0.905, weight="bold")
    fig.savefig(
        path_to_results
        / "xojn"
        / "ssn_mean_yz"
        / f"{sens.lower()}_{'CTRL'.lower()}_{var}_ssn_mean_yz_{layer}.png"
    )

In [ ]:
if yz:
    # Choose atmospheric layer
    layer = "0-17km"  # 0-2km 5-10km 0-17km 0-85km

    # Calculate 9-year-average zonal seasonal mean in a chosen atmospheric layer
    VRBLS = {}
    for exp in EXPERIMENTS.keys():
        VRBLS[exp] = {}
        for var in ["o3", "nox", "noy"]:  # vrbls[exp].keys():
            VRBLS[exp][var] = {}
            for ssn in SEASONS.keys():
                # Extract a time series for each latitude-longitude within a layer by season
                # and average each time slice in longitude
                ssn_ts = (
                    vrbls[exp][var]
                    .extract(LAYERS[layer]["iris_constr"] & SEASONS[ssn]["iris_constr"])
                    .collapsed(["longitude"], iris.analysis.MEAN)
                )
                # Calculate 9-year-average seasonal mean
                ssn_mean = ssn_ts.collapsed("year", iris.analysis.MEAN)
                # Find minimum and maximum
                ssn_mean_min, ssn_mean_max = ssn_mean.data.min(), ssn_mean.data.max()
                # Assemble data
                VRBLS[exp][var][ssn] = {
                    "ssn_ts": ssn_ts,
                    "ssn_mean": ssn_mean,
                    "ssn_mean_min": ssn_mean_min,
                    "ssn_mean_max": ssn_mean_max,
                }

    # Calculate seasonal mean differences and their statistical significance, and find extrema
    for sens in [k for k in EXPERIMENTS.keys() if k != "CTRL"]:
        for var in tqdm(VRBLS["CTRL"].keys()):
            for ssn in tqdm(SEASONS.keys(), leave=False):
                ssn_mean_diff_abs = (
                    VRBLS[sens][var][ssn]["ssn_mean"] - VRBLS["CTRL"][var][ssn]["ssn_mean"]
                )
                ssn_mean_diff_pct = ssn_mean_diff_abs / VRBLS["CTRL"][var][ssn]["ssn_mean"] * 100
                ssn_mean_diff_abs_min, ssn_mean_diff_abs_max = (
                    ssn_mean_diff_abs.data.min(),
                    ssn_mean_diff_abs.data.max(),
                )
                ssn_mean_diff_pct_min, ssn_mean_diff_pct_max = (
                    ssn_mean_diff_pct.data.min(),
                    ssn_mean_diff_pct.data.max(),
                )
                pval = calc_mean_diff_pval(
                    VRBLS["CTRL"][var][ssn]["ssn_ts"], VRBLS[sens][var][ssn]["ssn_ts"]
                )
                # Assemble data
                VRBLS[sens][var][ssn] = {
                    "ssn_mean_diff_abs": ssn_mean_diff_abs,
                    "ssn_mean_diff_pct": ssn_mean_diff_pct,
                    "pval": pval,
                    "ssn_mean_diff_abs_min": ssn_mean_diff_abs_min,
                    "ssn_mean_diff_abs_max": ssn_mean_diff_abs_max,
                    "ssn_mean_diff_pct_min": ssn_mean_diff_pct_min,
                    "ssn_mean_diff_pct_max": ssn_mean_diff_pct_max,
                }

    # Find common extrema
    var_lims = {}
    for var in ["o3", "nox", "noy"]:
        var_lims[var] = {}
        ssn_mean_min_com = []
        ssn_mean_max_com = []
        ssn_mean_diff_abs_min_com = []
        ssn_mean_diff_abs_max_com = []
        ssn_mean_diff_pct_min_com = []
        ssn_mean_diff_pct_max_com = []
        for ssn in SEASONS.keys():
            ssn_mean_min_com.append(VRBLS["CTRL"][var][ssn]["ssn_mean_min"])
            ssn_mean_max_com.append(VRBLS["CTRL"][var][ssn]["ssn_mean_max"])
            for sens in [k for k in EXPERIMENTS.keys() if k != "CTRL"]:
                ssn_mean_diff_abs_min_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_abs_min"])
                ssn_mean_diff_abs_max_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_abs_max"])
                ssn_mean_diff_pct_min_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_pct_min"])
                ssn_mean_diff_pct_max_com.append(VRBLS[sens][var][ssn]["ssn_mean_diff_pct_max"])
        var_lims[var] = {
            "ssn_mean_min_com": np.min(ssn_mean_min_com),
            "ssn_mean_max_com": np.max(ssn_mean_max_com),
            "ssn_mean_diff_abs_min_com": np.min(ssn_mean_diff_abs_min_com),
            "ssn_mean_diff_abs_max_com": np.max(ssn_mean_diff_abs_max_com),
            "ssn_mean_diff_pct_min_com": np.min(ssn_mean_diff_pct_min_com),
            "ssn_mean_diff_pct_max_com": np.max(ssn_mean_diff_pct_max_com),
        }

    # Plot
    ctrl_dict = VRBLS["CTRL"]
    for sens in [k for k in EXPERIMENTS.keys() if k != "CTRL"]:
        for var in ["o3", "nox"]:
            sens_dict = VRBLS[sens]
            plt_ssn_cross_sections(var, layer, ctrl_dict, sens_dict)